In [ ]:
from sage.sat.solvers.dimacs import DIMACS
from sage.all import *
from sage.combinat.posets.poset_examples import Posets
import itertools

In [ ]:
import sage.libs.ecl
sage.libs.ecl.ecl_eval("(ext:set-limit 'ext:heap-size 0)")

<ECL: 0>

In [ ]:
from enum import Enum

class VarType(Enum): 
    IsLess = 1
    IsMore = 2
    IsUsed = 3

In [ ]:
def construct_variable_map(poset, linear_extensions_count):
    '''
    is_lower[a, b, i]  > 0 means that a < b in the i-th linear extension
    is_higher[a, b, i] > 0 means that a > b in the i-th linear extension
    Is_used[a, i]      > 0 means that a is in the i-th linear extension
    '''
    variable_idx = 1
    
    is_lower = {}    
    for a,b in itertools.combinations(poset, int(2)):
        for i in range(linear_extensions_count):
            is_lower[a,b,i] = variable_idx
            variable_idx += 1

    is_higher = {}    
    for a,b in itertools.combinations(poset, int(2)):
        for i in range(linear_extensions_count):
            is_higher[a,b,i] = variable_idx
            variable_idx += 1

    is_used = {}    
    for a in poset:
        for i in range(linear_extensions_count):
            is_used[a, i] = variable_idx
            variable_idx += 1
    
    def is_less_variable_getter(*args):
        a, b, i = args
        if b < a:
            raise Exception("b < a")
        return is_lower[a,b,i]
    
    def is_more_variable_getter(*args):
        a, b, i = args
        if b < a:
            raise Exception("b < a")
        return is_higher[a,b,i]
    
    def is_used_variable_getter(*args):
        a, i = args
        return is_used[a, i]

    def variable_getter(VarType, *args):
        match VarType:
            case VarType.IsLess: return is_less_variable_getter(*args)
            case VarType.IsMore: return is_more_variable_getter(*args)
            case VarType.IsUsed: return is_used_variable_getter(*args)
            case _: raise Exception("Unknown variable type")
        
    return variable_getter

In [ ]:
def get_transitivity_clauses(poset, linear_extensions_count, var_getter):
    '''
    For each linear extension and each a, b, c that are used:
        (a > b or b > c or a < c)
    Because if a < b and b < c that implies a < c
    '''

    clauses = []

    for a, b, c in itertools.combinations(poset, int(3)):
        for i in range(linear_extensions_count):
            use_a = var_getter(VarType.IsUsed, a, i)
            use_b = var_getter(VarType.IsUsed, b, i)
            use_c = var_getter(VarType.IsUsed, c, i)
            a_less_b = var_getter(VarType.IsLess, a, b, i)
            b_less_c = var_getter(VarType.IsLess, b, c, i)
            a_less_c = var_getter(VarType.IsLess, a, c, i)
            clauses.append((-use_a, -use_b, -use_c, -a_less_b, -b_less_c, a_less_c))

    for a, b, c in itertools.combinations(poset, int(3)):
        for i in range(linear_extensions_count):
            use_a = var_getter(VarType.IsUsed, a, i)
            use_b = var_getter(VarType.IsUsed, b, i)
            use_c = var_getter(VarType.IsUsed, c, i)
            a_more_b = var_getter(VarType.IsMore, a, b, i)
            b_more_c = var_getter(VarType.IsMore, b, c, i)
            a_more_c = var_getter(VarType.IsMore, a, c, i)
            clauses.append((-use_a, -use_b, -use_c, -a_more_b, -b_more_c, a_more_c))

    return clauses

In [ ]:
def generate_use_clauses(poset, linear_extensions_count, local_dim, var_getter):
    '''
    1. a < b => a is used and b is used
    2. a > b => a is used and b is used 
    3. if a and b are used then a < b or a > b
    4. a is used at most local_dim times
    '''
    
    clauses = []

    for a, b in itertools.combinations(poset, int(2)):
        for i in range(linear_extensions_count):
            is_lower = var_getter(VarType.IsLess, a, b, i)
            is_used_a = var_getter(VarType.IsUsed, a, i)
            is_used_b = var_getter(VarType.IsUsed, b, i) 
            clauses.append((is_used_a, -is_lower))
            clauses.append((is_used_b, -is_lower))

    for a, b in itertools.combinations(poset, int(2)):
        for i in range(linear_extensions_count):
            is_higher = var_getter(VarType.IsMore, a, b, i)
            is_used_a = var_getter(VarType.IsUsed, a, i)
            is_used_b = var_getter(VarType.IsUsed, b, i)
            clauses.append((is_used_a, -is_higher))
            clauses.append((is_used_b, -is_higher))
            
    for a, b in itertools.combinations(poset, int(2)):
        for i in range(linear_extensions_count):
            is_used_a = var_getter(VarType.IsUsed, a, i)
            is_used_b = var_getter(VarType.IsUsed, b, i)
            is_lower = var_getter(VarType.IsLess, a, b, i)
            is_higher = var_getter(VarType.IsMore, a, b, i)
            clauses.append((-is_used_a, -is_used_b, is_lower, is_higher))

    cannot_be_used_times = local_dim + 1
    for a in poset:
        for x in itertools.combinations(range(linear_extensions_count), int(cannot_be_used_times)):
            clauses.append(tuple([-var_getter(VarType.IsUsed, a, i) for i in x]))

    return clauses

In [ ]:
def generate_poset_clauses(boolean_graph, linear_extensions_count, var_getter):
    '''
    1. Keep same order 
    2. If a || b then in one linear extension a < b and in another b < a
    '''
    
    clauses = []

    # For each a and b that a < b in Boolean Lattice there must be PLE that a < b 
    for a, b in sorted(boolean_graph.edges(labels=False)):
        clauses.append(tuple([var_getter(VarType.IsLess, a, b, i) for i in range(linear_extensions_count)]))
        for i in range(linear_extensions_count):
            clauses.append(tuple([-var_getter(VarType.IsMore, a, b, i)]))
                 

    # For each a and b that a || b in Boolean Lattice
    for a in sorted(boolean_graph.vertices()):
        for b in sorted(boolean_graph.vertices()):
            if a != b and a < b and not boolean_graph.has_edge(a, b):
                one_way = [var_getter(VarType.IsLess, a, b, i) for i in range(linear_extensions_count)]
                another_way = [var_getter(VarType.IsMore, a, b, i) for i in range(linear_extensions_count)]
                clauses.append(tuple(one_way))
                clauses.append(tuple(another_way))
        
    return clauses

In [ ]:
def generate_less_more_clauses(poset, linear_extensions_count, var_getter):
    '''
    1. a < b => !(b < a) and  b < a => !(a < b)
    '''
    
    clauses = []

    for a, b in itertools.combinations(poset, int(2)):
        for i in range(linear_extensions_count):
            is_lower = var_getter(VarType.IsLess, a, b, i)
            is_higher = var_getter(VarType.IsMore, a, b, i)
            clauses.append((-is_lower, -is_higher))
            
    return clauses

In [ ]:
def get_boolean_graph(dim):
    B = Posets.BooleanLattice(dim)
    return DiGraph([x for x in B.relations() if x[0] != x[1]])

In [ ]:
def generate_clauses(dim, linear_extensions_count, local_dim):
    boolean_graph = get_boolean_graph(dim)

    vertices = sorted(boolean_graph.vertices())
    var_getter = construct_variable_map(vertices, linear_extensions_count)

    clauses = get_transitivity_clauses(vertices, linear_extensions_count, var_getter) # done
    clauses += generate_poset_clauses(boolean_graph, linear_extensions_count, var_getter) # done
    clauses += generate_use_clauses(vertices, linear_extensions_count, local_dim, var_getter)
    clauses += generate_less_more_clauses(vertices, linear_extensions_count, var_getter)

    return clauses

def save_problem(dim, local_dim, linear_extensions_count, file_name):
    clauses = generate_clauses(dim, linear_extensions_count, local_dim)

    sat_generator = DIMACS()
    for c in clauses:
        sat_generator.add_clause(c)
    sat_generator.clauses(file_name)

In [ ]:
save_problem(
    dim = 7,
    local_dim = 5,
    linear_extensions_count = 7,
    file_name = "dim757.dimacs"
)

In [ ]:
def to_str(x):
    str = ''

    current_letter = 'a'
    current_number = 1

    while current_number <= x:
        if x & current_number:
            str += current_letter
        current_number *= 2
        current_letter = chr(ord(current_letter) + 1)      
     

    has_any_letter = any(c.isalpha() for c in str) 
    return str if has_any_letter else "(/)"

In [ ]:
from functools import cmp_to_key

def recover_order(values, variables, n, lin_ext_idx):
    is_used = lambda x: values[variables(VarType.IsUsed, x, lin_ext_idx)] > 0
    ordering = list(filter(is_used,[e for e in range(1 << n)]))
    

    def Compare(a, b):
        was_reversed = False
        if a > b:
            a, b = b, a
            was_reversed = True
        is_lower = values[variables(VarType.IsLess, a, b, lin_ext_idx)]
        is_higher = values[variables(VarType.IsMore, a, b, lin_ext_idx)]

        multiplier = 1 if not was_reversed else -1
        if is_lower:
            return -1 * multiplier
        if is_higher:
            return 1 * multiplier
        
    return sorted(ordering, key=cmp_to_key(Compare))

In [ ]:
def to_list(data):
    return [0] + sum(list(map(lambda x:  x.strip().split(' ')[1:], data.split("\n"))), [])

In [ ]:
def to_map(list_data):
    values = {i:int(list_data[i]) > 0 for i in range(len(list_data))}
    values.update({-i : not (int(list_data[i]) > 0) for i in range(len(list_data))})
    return values

In [ ]:
def parse_solution(n, le_count, data, should_to_string = False):
    variables = construct_variable_map(sorted(get_boolean_graph(n).vertices()), le_count)
    values = to_map(to_list(data))
    result = [recover_order(values, variables, n, j) for j in range(le_count)]
    if should_to_string:
        return [list(map(to_str, x)) for x in result]
    return result


In [ ]:
file = "75.out"
data = open(file, "r").read()

In [ ]:
parse_solution(7, 7, data, False)

[[88,
  2,
  10,
  32,
  104,
  56,
  120,
  42,
  1,
  9,
  25,
  33,
  41,
  57,
  26,
  58,
  3,
  35,
  19,
  73,
  51,
  11,
  27,
  43,
  59,
  74,
  106,
  90,
  122,
  83,
  89,
  75,
  113,
  115,
  105,
  91,
  121,
  107,
  123,
  4,
  68,
  6,
  70,
  20,
  84,
  22,
  86,
  12,
  28,
  5,
  69,
  13,
  76,
  77,
  36,
  100,
  14,
  30,
  92,
  21,
  7,
  85,
  78,
  15,
  79,
  87,
  29,
  31,
  94,
  95,
  37,
  44,
  45,
  38,
  46,
  39,
  47,
  52,
  60,
  54,
  61,
  62,
  63,
  108,
  110,
  101,
  109,
  111,
  116,
  119,
  124,
  127],
 [0,
  96,
  34,
  98,
  16,
  48,
  50,
  112,
  114,
  4,
  68,
  84,
  36,
  100,
  52,
  38,
  54,
  102,
  8,
  72,
  40,
  12,
  104,
  76,
  44,
  108,
  24,
  60,
  46,
  116,
  118,
  92,
  58,
  62,
  74,
  106,
  78,
  120,
  124,
  90,
  110,
  94,
  122,
  126,
  1,
  65,
  5,
  69,
  67,
  9,
  13,
  73,
  77,
  33,
  97,
  37,
  41,
  101,
  105,
  45,
  11,
  75,
  99,
  107,
  71,
  109,
  79,
  103,
  111,
  17,
 